<a href="https://colab.research.google.com/github/kurniayazid/Indonesia-geoscraping/blob/master/Scraping_kab_kot_geolocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Geolocation for Indonesia Kabupaten-Kota

## Description

~ ~ ~ ~

### 1. Installing & importing required libraries

In [9]:
# Installing libraries
!pip install geocoder
!pip install folium
!pip install tqdm

In [10]:
# Importing required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from tqdm import tqdm # from loading screen

print("Libraries imported.")

Libraries imported.
Libraries imported.


### 2. Importing dataset

In [0]:
# Dataset url from my repo
dataset_url = 'https://raw.githubusercontent.com/kurniayazid/Indonesia-geoscraping/master/region-code/code-area.csv'

# Convert csv to pandas dataframe
df = pd.read_csv(dataset_url)

In [12]:
# Check dataset
df.head()

,code,parent_code,name
0,1101,11,KABUPATEN SIMEULUE
1,1102,11,KABUPATEN ACEH SINGKIL
2,1103,11,KABUPATEN ACEH SELATAN
3,1104,11,KABUPATEN ACEH TENGGARA
4,1105,11,KABUPATEN ACEH TIMUR


,code,parent_code,name
0,1101,11,KABUPATEN SIMEULUE
1,1102,11,KABUPATEN ACEH SINGKIL
2,1103,11,KABUPATEN ACEH SELATAN
3,1104,11,KABUPATEN ACEH TENGGARA
4,1105,11,KABUPATEN ACEH TIMUR


In [13]:
# Total collected municipalities and regencies

print("In total, this dataset contains:", df.shape[0], 'municipalities and regencies')

In total, this dataset contains: 514 municipalities and regencies
In total, this dataset contains: 514 municipalities and regencies


## Retrieving geolocations (Longitudes & Latitudes)

In [0]:
# define a function to get location coordinates
geolocator = Nominatim(user_agent="my-application")
def get_latlng(munreg):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while lat_lng_coords is None:
        g = geocoder.arcgis('{}'.format(munreg))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(munreg) for munreg in tqdm(df["name"].tolist()) ]

100%|██████████| 514/514 [05:19<00:00,  1.59it/s]


In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [21]:
df.head()

,code,parent_code,name,Latitude,Longitude
0,1101,11,KABUPATEN SIMEULUE,2.6199,96.0790
1,1102,11,KABUPATEN ACEH SINGKIL,2.5378,97.9366
2,1103,11,KABUPATEN ACEH SELATAN,3.1650,97.4441
3,1104,11,KABUPATEN ACEH TENGGARA,3.3695,97.7084
4,1105,11,KABUPATEN ACEH TIMUR,4.6542,97.6126


In [22]:
df.shape

(514, 5)

In [23]:
df

,code,parent_code,name,Latitude,Longitude
0,1101,11,KABUPATEN SIMEULUE,2.619900e+00,96.079000
1,1102,11,KABUPATEN ACEH SINGKIL,2.537800e+00,97.936600
2,1103,11,KABUPATEN ACEH SELATAN,3.165000e+00,97.444100
3,1104,11,KABUPATEN ACEH TENGGARA,3.369500e+00,97.708400
4,1105,11,KABUPATEN ACEH TIMUR,4.654200e+00,97.612600
5,1106,11,KABUPATEN ACEH TENGAH,4.510000e+00,96.855000
6,1107,11,KABUPATEN ACEH BARAT,4.454400e+00,96.178400
7,1108,11,KABUPATEN ACEH BESAR,5.367200e+00,95.531200
8,1109,11,KABUPATEN PIDIE,5.381110e+00,95.918350
9,1110,11,KABUPATEN BIREUEN,5.093000e+00,96.595600


In [0]:
df.to_csv("indo-munreg-geolocations.csv", index=False)